## Импорт библиотек

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import sys
from glob import glob
from os.path import join
import os
import re
import multiprocessing as mp
from joblib import Parallel, delayed
import numpy as np
%pip install lxml

np.random.seed(42)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


## Парсинг телеграм чата

In [16]:
def get_str_from_file(path_to_file: str) -> str:
    '''Функция для полуения строки из текстового файла'''
    with open(path_to_file) as f:
        contents = f.readlines()
    text = ' '.join(content.strip() for content in contents).strip()
    # удаляем ссылки из сообщений
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

def read_html(path_to_html: str):
    '''Функция для чтения html'''
    with open(path_to_html, "r") as s:    
        contents = s.read() 
    soup = BeautifulSoup(contents, 'html.parser')    
    return soup

def save_msg_to_txt(soup, msg_id: str, output_path: str):
    '''Функция для сохранения сообщения в текстовый файл'''
    with open(output_path, 'w') as f:
        try:
            f.write(soup.body.find('div', id=msg_id).find_all('div', attrs={'class':"text"})[0].text)
        except IndexError:
            pass 

def create_df_from_dict(dictionary: dict, output_dir: str, df_name:str): 
    '''Функция преобразования словаря в DataFrame'''
    if len(dictionary) != 0:                         
        df = pd.DataFrame({
            'quest_id' : list(dictionary.keys()),
            'ans_id': list(dictionary.values())})
        df.to_csv(join(output_dir, df_name), index=False)

def parse_html(path_to_html: str):
    '''Функция для парсинга html файла'''
    
    # считаем html-документ
    soup = read_html(path_to_html)
    
    # словарь для хранения связей вопрос-ответ
    this_html_connect_dict = {}
     
    file_name = path_to_html.split('/')[-1].split('.')[0]
    connections_dir = join('..', 'data', 'connections_info')
    output_dir = join('..', 'data', 'connections_info', file_name)
    messages_dir = join('..', 'data', 'messages_text')
    
    # получим id всех сообщений в html-документе
    msg_ids = [tag['id'] for tag in soup.select('div[id]')]
    
    # создаем необходимые папки
    os.makedirs(messages_dir, exist_ok=True)
    os.makedirs(connections_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    with open(join(output_dir, file_name + '_info.txt'), 'w') as f:
        for elem in msg_ids:
            f.writelines(elem+'\n')
            
    for msg_id in msg_ids:
        # проверяем содержит ли msg_id в себе тег 'a' 
        a_list = soup.body.find('div', id=msg_id).find_all('a')
        if a_list != []:
            for a_elem in a_list:
                # если в тексте тега 'а' msg_id есть фраза 'this message', 
                # значит это сообщение является ответом на какое-то другое сообщение
                if a_elem.text == 'this message':
                    # получим id сообщения, на которое отвечали
                    question_id = 'message' + re.findall("\d+", a_elem.get('href'))[0]
                    # проверим есть ли сообщение, на которое отвечали (question_id), в текущем html-файле
                    if question_id in msg_ids:
                        quest_txt_path = join(messages_dir, question_id + '.txt')
                        ans_txt_path = join(messages_dir, msg_id + '.txt')
                        # создаем текстовый файл, только если его еще нет
                        if not os.path.exists(quest_txt_path):
                            save_msg_to_txt(soup=soup, msg_id=question_id, output_path=quest_txt_path)
                                 
                        if not os.path.exists(ans_txt_path):
                            save_msg_to_txt(soup=soup, msg_id=msg_id, output_path=ans_txt_path)
                            
                        if (os.path.exists(quest_txt_path) + os.path.exists(ans_txt_path)) == 2:
                            this_html_connect_dict[question_id] = msg_id

    # создаем df содержащий связи вопрос-ответ
    create_df_from_dict(dictionary=this_html_connect_dict,
                        output_dir=output_dir,
                        df_name='this_html_connect.csv')
    
def get_qa_txt_file(df_path: str, msg_dir: str, output_path: str):
    '''
    Функция создания текстового файла в формате:
    
    Q: ##текст вопроса##
    A: ##текст ответа##
    '''
    df = pd.read_csv(df_path)
    with open(output_path, "w") as outfile:
        for q, a in zip(df.quest_id.values,
                        df.ans_id.values):
            quest_path = join(msg_dir, q+'.txt')
            ans_path = join(msg_dir, a+'.txt')
            # проверяем существование txt файлов, а также наличие вопроса
            if ((os.path.exists(quest_path) + os.path.exists(ans_path)) == 2) and \
               (get_str_from_file(quest_path).find('?') != -1) and \
               (len(get_str_from_file(ans_path)) > 3):
                outfile.write('Q: ' + get_str_from_file(quest_path).strip() + '\n')
                outfile.write('A: ' + get_str_from_file(ans_path).strip() + '\n')

def create_dataset(list_of_txt: list, output_path: str):
    '''Функция создания датасета из нескольких текстовых файлов'''
    with open(output_path, 'w') as outfile:
        for fname in list_of_txt:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

### Выполняем парсинг тг чата

In [41]:
#!c1.8
%%time
sys.setrecursionlimit(10000)
from multiprocessing import Pool
html_list = glob(join('..', 'data', 'ChatExport_Alice', '*.html'))
with Pool(8) as p:
    p.map(parse_html, html_list)

CPU times: user 8.54 s, sys: 3.83 s, total: 12.4 s
Wall time: 1h 16min 8s


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:840: UserWarning: The following variables cannot be serialized: p
  warnings.warn(message)


In [44]:
#!c1.8
len(html_list)

359

In [40]:
#!c1.8
html_list[0]

'../data/ChatExport_Alice/messages10.html'

In [63]:
#!c1.8
%%time
messages_dir = join('..', 'data', 'messages_text')
connections_path = join('..', 'data', 'connections_info')
dir_list = [join(connections_path, directory) for directory in os.listdir(connections_path)]
for dir_path in dir_list:
    get_qa_txt_file(join(dir_path, 'this_html_connect.csv'), 
                    messages_dir,
                    join(dir_path, dir_path.split('/')[-1] + '.txt'))

CPU times: user 20.8 s, sys: 21.3 s, total: 42.1 s
Wall time: 3min 49s


### Создаем датасет


In [62]:
#!c1.8
%%time
txt_list = [join(dir_path, dir_path.split('/')[-1] + '.txt') for dir_path in dir_list]
train_size = round(len(txt_list)*0.8)
create_dataset(list_of_txt=txt_list[:train_size], output_path=join('..', 'data','train_house.txt'))
create_dataset(list_of_txt=txt_list[train_size:], output_path=join('..', 'data','valid_house.txt'))

CPU times: user 128 ms, sys: 68 ms, total: 196 ms
Wall time: 1.74 s


In [ ]:
#!c1.8
